In [ ]:
import pycuda.autoinit
import numpy as np
from pycuda import gpuarray
import pycuda.driver as drv
import pandas as pd
from pycuda.compiler import SourceModule
import cv2

In [ ]:
# Compile cuda code
module = SourceModule(open("kernels_simple_spheres.cu", "r").read())
kernel_simple_spheres = module.get_function("simple_spheres")

In [ ]:
# Define spheres
spheres_coord = np.array([[100, 300, 10],
                           [200, 30, 30]], dtype=np.float32)
spheres_radius = np.array([50, 50], dtype=np.float32)
spheres_color = np.array([[255, 0, 0],
                          [0, 0, 255]], dtype=np.float32)
n_spheres = len(spheres_coord)

spheres_coord_gpu = gpuarray.to_gpu(spheres_coord)
spheres_radius_gpu = gpuarray.to_gpu(spheres_radius)
spheres_color_gpu = gpuarray.to_gpu(spheres_color)

In [ ]:
# Define image
dim = 512
bitmap = np.zeros((dim, dim, 3), dtype=np.int32)
bitmap_gpu = gpuarray.to_gpu(bitmap)

In [ ]:
# Calculate the bitmap
grid_size = 16
block = (dim//grid_size, dim//grid_size, 1)
grid = (grid_size, grid_size)

kernel_simple_spheres(bitmap_gpu, spheres_coord_gpu, spheres_radius_gpu, spheres_color_gpu,
                      np.int32(dim), np.int32(n_spheres),
                      block=block,
                      grid=grid)

In [ ]:
image = cv2.cvtColor(bitmap_gpu.get().astype(np.uint8), cv2.COLOR_RGB2BGR)
cv2.imwrite('color_img.jpg', image)